In [ ]:
from data.overnight.domain.domain_base import Domain
import re
import tempfile
import subprocess

In [ ]:
overnight_domains = ['basketball', 'blocks', 'calendar', 'housing', 'publications', 'recipes', 'restaurants', 'socialnetwork']

In [ ]:
def read_data(path, domain_idx):
    ex_list = []
    with open(path, 'r') as infile:
        lines = [line.strip() for line in infile]
        for i in range(len(lines))[::8]:
            ex = {}
            ex['q'] = re.findall(r'''\"(.*?)\"''',lines[i+1])[0]
            ex['tir'] = re.findall(r'''\"(.*?)\"''',lines[i+2])[0]
            ex['lamb'] = lines[i+4].strip()
            ex['domain'] = domain_idx
            ex_list.append(ex)
    return ex_list

In [ ]:
ex_list = []
for idx, domain in enumerate(overnight_domains):
    ex_list += read_data('../sempre/lib/data/overnight/{}.paraphrases.test.examples'.format(domain), idx)
# test_inputs = [i['q'] for i in ex_list]
# test_targets = [i['tir'] for i in ex_list]
# test_lambs = [i['lamb'] for i in ex_list]
test_domains = [i['domain'] for i in ex_list]

In [ ]:
test_preds = []
test_golds = []
with open("./lambda_results.txt") as f:
    for line in f:
        test_preds.append(line.strip())
with open("./lambda_golds.txt") as f:
    for line in f:
        test_golds.append(line.strip())

In [ ]:
def read_data(path):
    ex_list = []
    with open(path, 'r') as infile:
        for line in infile:
            line = line.strip()
            if line == '':
                continue
            q, lf = line.split('\t')
            ex_list.append({"q": q.strip(), "lf": lf.strip()})
    return ex_list

In [ ]:
test_gold = []
for domain in overnight_domains:
    test_gold += [i['lf'] for i in read_data("./data/overnight/dataset/{}_test.tsv".format(domain))]

In [ ]:
wrong_pairs = []
count = 0
assert len(test_domains) == len(test_preds) == len(test_golds) == len(test_gold)
for p, g, i, d in zip(test_preds, test_golds, test_inputs, test_domains):
    # p = p.replace("!", " !").replace(".size", " .size")
    if p != g:
        print(len(wrong_pairs))
        wrong_pairs.append((d, [p,g]))
        print("input:\t", i)   
        print("pred:\t", p)
        print("gold:\t", g)
        print("\n")
# print("wrong_pairs:", len(wrong_pairs))
# print("acc:", 1 - len(wrong_pairs) / len(test_preds))

In [ ]:
def format_overnight(lf):
    replacements = [
        ('(', ' ( '), # make sure ( and ) must have blank space around
        (')', ' ) '),
        ('! ', '!'),
        ('SW', 'edu.stanford.nlp.sempre.overnight.SimpleWorld'),
    ]
    for a, b in replacements:
        lf = lf.replace(a, b)
    # remove redundant blank spaces
    lf = re.sub(' +', ' ', lf)
    return lf.strip()

def normalize(lf_list):
    return [format_overnight(lf) for lf in lf_list]

def obtain_denotations(lf_list, domain):
    tf = tempfile.NamedTemporaryFile('w+t', encoding='utf8', suffix='.examples')
    for line in lf_list:
        tf.write(line + '\n')
    tf.flush()
    msg = subprocess.check_output(['./evaluator/overnight', domain, tf.name])
    msg = msg.decode('utf8')
    tf.close()
    denotations = [
        line.split('\t')[1] for line in msg.split('\n')
        if line.startswith('targetValue\t')
    ]
    return denotations

In [ ]:
def is_valid(ans):
    return not ('BADJAVA' in ans or 'ERROR' in ans or 'Exception' in ans or ans == 'null')

In [ ]:
normalize(x[1])

In [ ]:
x = wrong_pairs[505]
ans = obtain_denotations(normalize(x[1]), overnight_domains[x[0]])
print(ans[0])
print(ans[1])